<a href="https://colab.research.google.com/github/kridtapon/Confluence-Breakout-System/blob/main/Confluence_Breakout_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: numba
    Found existing installation: numba 0.60.0
    Uninstalling numba-0.60.0:
      Successfully uninst

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Exponential Moving Averages (EMAs) with reusable parameters
def calculate_emas(df, close_col='Close', periods=None):

    if periods is None:
        periods = [20, 50, 100, 200]  # Default periods if none are provided

    for period in periods:
        df[f'EMA_{period}'] = df[close_col].ewm(span=period, adjust=False).mean()
    return df

# Function to detect confluence zone with reusable periods
def detect_confluence(df, periods=None):

    if periods is None:
        periods = [20, 50, 100, 200]  # Default periods if none are provided

    ema_columns = [f'EMA_{period}' for period in periods]
    df['Confluence_Upper'] = df[ema_columns].max(axis=1)
    df['Confluence_Lower'] = df[ema_columns].min(axis=1)
    return df

# Define stock symbol and time period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate EMAs with dynamic periods
df = calculate_emas(df, periods=[10, 30, 50, 100])  # Example of custom periods

# Detect Confluence Zone with dynamic periods
df = detect_confluence(df, periods=[10, 30, 50, 100])  # Example of custom periods

# Define Breakout Entry Conditions
df['High_Volume'] = df['Volume'] > df['Volume'].rolling(20).mean() * 1.5  # High volume condition
df['Breakout_Up'] = (df['Close'] > df['Confluence_Upper']) & df['High_Volume']
df['Breakout_Down'] = (df['Close'] < df['Confluence_Lower']) & df['High_Volume']

df['Entry'] = df['Breakout_Up']
df['Exit'] = df['Close'] < df['Confluence_Lower']  # Exit if price drops below confluence zone

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()



[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2019-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1510
Start Value                              100000.0
End Value                           345081.153499
Total Return [%]                       245.081153
Benchmark Return [%]                   331.537472
Max Gross Exposure [%]                      100.0
Total Fees Paid                       4620.011585
Max Drawdown [%]                        32.837916
Max Drawdown Duration                       353.0
Total Trades                                   12
Total Closed Trades                            11
Total Open Trades                               1
Open Trade PnL                       13908.321003
Win Rate [%]                            54.545455
Best Trade [%]                          88.028047
Worst Trade [%]                          -14.6176
Avg Winning Trade [%]                   31.393776
Avg Losing Trade [%]                    -6.170748


In [ ]:
apply this trading strategy to walk forward optimize code above
replace Ichimoku Cloud with EMA and confluence zone instead
using this parameter
MA_1=range(5,21)
MA_2=range(21,51)
MA_3=range(51,101)
MA_4=range(101,201)